In [1]:
%%capture
# !conda install -n base ipykernel --update-deps --force-reinstall -y
# !conda install pandas numpy -y
# !conda install -c plotly plotly=5.24.1 -y
# !pip install requests
# !pip install nbformat

In [2]:
import pandas as pd
import numpy as np
import requests
import os

In [3]:
key = os.getenv('openchargemap_key')

In [4]:
# API endpoint
url = "https://api.openchargemap.io/v3/poi/"

# Parameters for the API request
params = {
    'output': 'json',
    'countrycode': 'AU',
    'maxresults': 10000,
    'key': 'bed3bd25-94b2-419f-8ce7-861f59c4e57c'
}

# Making the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Load the JSON response
    data = response.json()
    
    # Convert the JSON data to a pandas DataFrame
    df = pd.json_normalize(data)
    
    # Display the first few rows of the DataFrame
else:
    print(f"Failed to retrieve data: {response.status_code}")


In [5]:
df = df.drop(columns=['UserComments', 'PercentageSimilarity', 'MediaItems', 'DateLastStatusUpdate', 'DataQualityLevel', 'UUID', 
                      'ParentChargePointID', 'OperatorID', 'UsageTypeID', 'SubmissionStatusTypeID', ])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 74 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   IsRecentlyVerified                                     1125 non-null   bool   
 1   DateLastVerified                                       1125 non-null   object 
 2   ID                                                     1125 non-null   int64  
 3   DataProviderID                                         1125 non-null   int64  
 4   DataProvidersReference                                 6 non-null      object 
 5   OperatorsReference                                     50 non-null     object 
 6   UsageCost                                              862 non-null    object 
 7   Connections                                            1125 non-null   object 
 8   NumberOfPoints                                  

In [7]:
df = df.drop(columns=df.filter(like='.ID', axis=1).columns)

In [8]:
connections = df['Connections'].apply(pd.Series)
df = df.drop(columns=['Connections'],axis=0)

In [9]:
connections = pd.json_normalize(connections[0])

In [10]:
connections.columns

Index(['ID', 'ConnectionTypeID', 'Reference', 'StatusTypeID', 'LevelID',
       'Amps', 'Voltage', 'PowerKW', 'CurrentTypeID', 'Quantity', 'Comments',
       'ConnectionType.FormalName', 'ConnectionType.IsDiscontinued',
       'ConnectionType.IsObsolete', 'ConnectionType.ID',
       'ConnectionType.Title', 'StatusType.IsOperational',
       'StatusType.IsUserSelectable', 'StatusType.ID', 'StatusType.Title',
       'Level.Comments', 'Level.IsFastChargeCapable', 'Level.ID',
       'Level.Title', 'CurrentType.Description', 'CurrentType.ID',
       'CurrentType.Title', 'CurrentType', 'Level', 'StatusType'],
      dtype='object')

In [11]:
final_df = pd.merge(df, connections)

In [12]:
final_df = final_df[["ID","IsRecentlyVerified","UsageCost",
"NumberOfPoints","DateCreated",
"DataProvider.WebsiteURL","DataProvider.Title",
"OperatorInfo.IsRestrictedEdit",
"OperatorInfo.Title","UsageType.IsPayAtLocation",
"UsageType.IsMembershipRequired",
"UsageType.IsAccessKeyRequired",
"UsageType.Title","StatusType.IsOperational",
"StatusType.IsUserSelectable",
"StatusType.Title","SubmissionStatus.IsLive",
"AddressInfo.Title","AddressInfo.StateOrProvince",
"AddressInfo.Postcode","AddressInfo.Latitude","AddressInfo.Longitude",
"AddressInfo.DistanceUnit","Reference",
"Amps","Voltage","PowerKW","Quantity",
"ConnectionType.FormalName","ConnectionType.IsDiscontinued",
"ConnectionType.IsObsolete","ConnectionType.Title",
"Level.IsFastChargeCapable","Level.Title",
"CurrentType.Description","CurrentType.Title",]]

In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              0 non-null      int64  
 1   IsRecentlyVerified              0 non-null      bool   
 2   UsageCost                       0 non-null      object 
 3   NumberOfPoints                  0 non-null      float64
 4   DateCreated                     0 non-null      object 
 5   DataProvider.WebsiteURL         0 non-null      object 
 6   DataProvider.Title              0 non-null      object 
 7   OperatorInfo.IsRestrictedEdit   0 non-null      object 
 8   OperatorInfo.Title              0 non-null      object 
 9   UsageType.IsPayAtLocation       0 non-null      object 
 10  UsageType.IsMembershipRequired  0 non-null      object 
 11  UsageType.IsAccessKeyRequired   0 non-null      object 
 12  UsageType.Title                 0 non-null      

In [14]:
import plotly.express as px

In [15]:
fig1 = px.bar(final_df, x='UsageType.Title', y='NumberOfPoints', color='UsageType.Title', 
              title='Number of Charging Points by Usage Type')
fig1.show()

In [16]:
fig2 = px.histogram(final_df, x='PowerKW', nbins=20, title='Distribution of Power (KW)')
fig2.show()

In [17]:
fig3 = px.bar(df.groupby('AddressInfo.StateOrProvince').NumberOfPoints.mean().reset_index(), 
              x='AddressInfo.StateOrProvince', y='NumberOfPoints', 
              title='Average Number of Points by State/Province')
fig3.show()

In [18]:
fig4 = px.pie(df, names='IsRecentlyVerified', title='Recently Verified Stations')
fig4.show()

In [19]:
fig5 = px.scatter_mapbox(
    final_df,
    lat="AddressInfo.Latitude",
    lon="AddressInfo.Longitude",
    hover_name="AddressInfo.StateOrProvince",
    hover_data=["NumberOfPoints"],
    color="NumberOfPoints",  # Color based on the number of points
    size="NumberOfPoints",  # Size of the marker based on the number of points
    zoom=3,
    mapbox_style="open-street-map"  # You can change this to 'satellite', 'stamen-terrain', etc.
)
fig5.show()

In [20]:
# final_df.to_csv('final.csv', index=False)